In [1]:
import gpytorch
import pyreadr
import pooling_params as gtp
import run_gpytorchkernel
import simple_bandits
from sklearn import preprocessing
import R_to_python_functions
import json

In [2]:
import numpy as np

In [3]:
import os

In [7]:
def main(data_path):
    result = pyreadr.read_r(data_path)
    baseline_features = ['temperature', 'logpresteps', 'sqrt.totalsteps',\
                         'dosage', 'engagement', 'work.location', 'other.location', 'variation']
    responsivity_features = ['dosage', 'engagement', 'work.location', 'other.location', 'variation']
    data_dict = process_data(result['train.dat'],baseline_features)
    standard_x = get_standard_x(data_dict['data'])

'/pooling-service/sabina_tmp/in_progress'

In [4]:
data_path = '../../data/train_EB.RData'

In [17]:
baseline_features = ['temperature', 'logpresteps', 'sqrt.totalsteps',\
                         'dosage', 'engagement', 'work.location', 'other.location', 'variation']
responsivity_features = ['dosage', 'engagement', 'work.location', 'other.location', 'variation']

In [32]:
def get_standard_x(X):
    return preprocessing.StandardScaler().fit_transform(X)

In [33]:
standard_x = get_standard_x(data_dict['data'])

In [5]:
def fake_data(n):
    #temperature, last steps, prior,location
    return [[np.random.random()*50+10,np.random.random()*30,np.random.random()*20,int(np.random.random()>.5)] for i in range(n)]

In [6]:
data = fake_data(100)

In [92]:
def initialize_policy_params_TS(standardize=False,baseline_features=None,psi_features=None,\
                                responsivity_keys=None):
    #,'location_1','location_2','location_3'
    #'continuous_temp',
    global_p =gtp.TS_global_params(21,baseline_features=baseline_features,psi_features=psi_features, responsivity_keys= responsivity_keys)
    #personal_p = pp.TS_personal_params()
    #global_p =gtp.TS_global_params(10,context_dimension)
    
    
    
    #global_p.mu_dimension = 64
    
    global_p.kdim =24
    #194
    global_p.baseline_indices = [i for i in range(3+ len(baseline_features)+2*len(responsivity_keys))]
    #[i for i in range(192)]
    #[0,1,2,3,4,5,6]
    global_p.psi_indices = [0] + [1+baseline_features.index(j) for j in psi_features] \
    + [len(baseline_features)+1] + [(2+len(baseline_features))+baseline_features.index(j) for j in psi_features]
    #[0,64]
    global_p.user_id_index =0
    
    global_p.psi_features =psi_features
    #[0,64]
    
    #print(global_p.psi_indices )
    
    #global_p.update_period = update_period
    
    global_p.standardize = standardize
    global_p.lr = .01

    initial_context = [0 for i in range(global_p.theta_dim)]
    
    global_p.mus0= global_p.get_mu0(initial_context)
    #global_p.get_mu0(initial_context)
    global_p.mus1= global_p.get_mu1(global_p.num_baseline_features)
    global_p.mus2= global_p.get_mu2(global_p.num_responsivity_features)
    #np.array([.120,3.3,-.11])
    #global_p.get_mu2(global_p.num_responsivity_features)
    
    #global_p.sigmas0= global_p.get_asigma(len( personal_p.mus0[person]))
    global_p.sigmas1= global_p.get_asigma(global_p.num_baseline_features+1)
    global_p.sigmas2= global_p.get_asigma( global_p.num_responsivity_features+1)
    

    global_p.mu2_knot = np.array([0]+[0 for i in range(global_p.num_responsivity_features)])
    global_p.mu1_knot = np.zeros(global_p.num_baseline_features+1)
    global_p.sigma1_knot = np.eye(global_p.num_baseline_features+1)
    global_p.sigma2_knot = np.eye(global_p.num_responsivity_features+1)

    return global_p

In [96]:
def get_phi(standard_x,all_dict,baseline_indices,responsivity_indices):
    ##returns users, transformed data, and transformed rewards
    to_return = []
    
    users = []
    for i in range(len(standard_x)):
        if all_dict['avail'][i]:
            raw_data = standard_x[i]
            temp = [1]
            temp.extend([raw_data[i] for i in baseline_indices])
            temp_two  = [1]
            temp_two.extend([raw_data[i] for i in responsivity_indices])
            temp.extend([all_dict['prob'][i]*d for d in temp_two])
            temp.extend([(all_dict['actions'][i]-all_dict['prob'][i])*d for d in temp_two])
            to_return.append(temp)
            users.append(all_dict['users'][i])
    to_adjust = [np.array(all_dict['reward']).mean()]+[0]*(len(to_return[0])-1)
    #to_adjust = np.ones(len(to_return[0]))
    y = np.array([all_dict['reward'][i]-np.dot(to_return[i],to_adjust) for i in range(len(to_return))])
    return np.array(to_return),y,np.array(users)

In [78]:
data_dict = process_data(result['train.dat'],baseline_features)
standard_x = get_standard_x(data_dict['data'])

In [97]:
X,y,users = get_phi(standard_x,data_dict,[i for i in range(len(baseline_features))],\
                    [i for i in range(baseline_features.index(responsivity_features[0]),len(baseline_features))])

In [81]:
np.array(data_dict['reward']).mean()
y.std()

1.5933105218817263

In [98]:
glob = initialize_policy_params_TS(standardize=False,baseline_features=[i for i in range(len(baseline_features))],psi_features=[],responsivity_keys=[i for i in range(3,len(baseline_features))])
               

In [99]:
def new_form(X):
    to_return = []
    for x in X:
        intercept = [1]
        base = intercept+x
        second = [.6*base[i] for i in range(len(base)) ]
        third = [int(np.random.random()>.5)-.6*base[i] for i in range(len(base)) ]
        to_return.append(base+second+third)
    return to_return
    
    

In [101]:
len(users)

13

In [94]:
glob.baseline_indices

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [102]:
temp_params = run_gpytorchkernel.run(X, users,y,glob)

In [104]:
inv_term = simple_bandits.get_inv_term(temp_params['cov'],X.shape[0],temp_params['noise'])
glob.update_params(temp_params)
glob.inv_term=inv_term

In [105]:
def get_posterior_parameters_all():
    temp = simple_bandits.calculate_posterior_faster(glob,\
                                                         participant.pid,participant.current_day_counter,\
                                                         history[0], history[1],history[2] )

In [107]:
temp = simple_bandits.calculate_posterior_faster(glob,\
                                                         1,0,\
                                                        X, users,np.array([[i] for i in y]) )

In [108]:
temp

(array([-0.18712527,  0.32614318,  0.        , -0.06628575,  0.        ,
         0.93582842]),
 [array([ 0.54548707, -0.18460375,  0.        ,  0.00227221,  0.        ,
          0.12541985]),
  array([ -1.84603747e-01,   8.54974941e-01,   0.00000000e+00,
           1.06562680e-04,   0.00000000e+00,   1.93457569e-02]),
  array([ 0.,  0.,  1.,  0.,  0.,  0.]),
  array([  2.27220866e-03,   1.06562680e-04,   0.00000000e+00,
           1.10233751e+00,   0.00000000e+00,  -1.16999239e-03]),
  array([ 0.,  0.,  0.,  0.,  1.,  0.]),
  array([ 0.12541985,  0.01934576,  0.        , -0.00116999,  0.        ,
          0.67793525])])